In [8]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()


# 현재 파일의 위치를 기준으로 프로젝트 루트(backend) 경로를 계산하여 추가
root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if root_path not in sys.path:
    sys.path.append(root_path)

In [ ]:
from datetime import datetime

from fastapi import APIRouter, Depends, HTTPException, status
from fastapi.responses import StreamingResponse
from sqlalchemy.orm import Session

from app.agents.multi_agent_graph import graph as multi_agent_graph
from app.db import get_db
from app.deps import get_current_user
from app.models import Conversation, Message, User
from app.schemas import (
    ChatResponse,
    ConversationCreate,
    ConversationOut,
    MessageCreate,
    MessageOut,
)

router = APIRouter(prefix="/conversations", tags=["conversations"])

In [ ]:
from app.agents.multi_agent_graph import graph as multi_agent_graph

async def test_streaming():
    async for event in multi_agent_graph.astream_events(
        {"query": "안녕", "chat_history": []}, version="v2"
    ):
        if event["event"] == "on_chat_model_stream":
            metadata = event.get("metadata", {})
            # structured output (router) 이벤트 건너뛰기
            if metadata.get("langgraph_node") not in [
                "generate",
                "llm",
                "calculate_tax_rate",
            ]:
                continue


await test_streaming()

[ChatOpenAI] ''
[ChatOpenAI] '{"'
[ChatOpenAI] 'target'
[ChatOpenAI] '":"'
[ChatOpenAI] 'll'
[ChatOpenAI] 'm'
[ChatOpenAI] '"}'
[ChatOpenAI] ''
[ChatOpenAI] ''
[ChatOpenAI] ''
[ChatOpenAI] ''
[ChatOpenAI] ''
[ChatOpenAI] ''
[ChatOpenAI] '안'
[ChatOpenAI] '녕하세요'
[ChatOpenAI] '!'
[ChatOpenAI] ' 무엇'
[ChatOpenAI] '을'
[ChatOpenAI] ' 도'
[ChatOpenAI] '와'
[ChatOpenAI] '드'
[ChatOpenAI] '릴'
[ChatOpenAI] '까요'
[ChatOpenAI] '?'
[ChatOpenAI] ' 필요'
[ChatOpenAI] '하'
[ChatOpenAI] '신'
[ChatOpenAI] ' 게'
[ChatOpenAI] ' 있'
[ChatOpenAI] '으면'
[ChatOpenAI] ' 말씀'
[ChatOpenAI] '해'
[ChatOpenAI] ' 주세요'
[ChatOpenAI] '.'
[ChatOpenAI] ' 예'
[ChatOpenAI] '를'
[ChatOpenAI] ' 들'
[ChatOpenAI] '면'
[ChatOpenAI] ':\n'
[ChatOpenAI] '-'
[ChatOpenAI] ' 한국'
[ChatOpenAI] '어'
[ChatOpenAI] '/'
[ChatOpenAI] '영'
[ChatOpenAI] '어'
[ChatOpenAI] ' 대'
[ChatOpenAI] '화'
[ChatOpenAI] ' 연'
[ChatOpenAI] '습'
[ChatOpenAI] '\n'
[ChatOpenAI] '-'
[ChatOpenAI] ' 정보'
[ChatOpenAI] ' 검색'
[ChatOpenAI] '이나'
[ChatOpenAI] ' 요'
[ChatOpenAI] '약'
[ChatOpenAI] 

### 한줄 코드

In [1]:
import sys
import os
from pathlib import Path

# 1. 프로젝트 루트(backend 폴더)를 sys.path에 추가
# 현재 위치: .../backend/app/routers/chat_test.ipynb
# 목표 위치: .../backend
current_dir = Path(os.getcwd())  # 또는 Path().resolve()
project_root = current_dir.parent.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 2. Import
from app.agents.multi_agent_graph import graph as multi_agent_graph


# 3. 비동기 스트리밍 테스트 함수
async def test_streaming():
    print("🔴 스트리밍 시작...\n" + "=" * 20)

    async for event in multi_agent_graph.astream_events(
        {"query": "소득세가 뭐야?", "chat_history": []}, version="v2"
    ):
        # 스트리밍 이벤트 감지
        if event["event"] == "on_chat_model_stream":
            metadata = event.get("metadata", {})

            if metadata.get("langgraph_node") not in [
                "generate",
                "llm",
                "calculate_tax_rate",
            ]:
                continue
            if "hallucination_check" in event.get("tags", []):
                continue

            # 청크 데이터 추출 및 출력
            chunk = event["data"]["chunk"]
            if chunk and hasattr(chunk, "content"):
                # end="" 옵션으로 줄바꿈 없이 이어서 출력 (타자기 효과)
                print(chunk.content, end="", flush=True)

    print("\n" + "=" * 20 + "\n🔵 스트리밍 종료")


# 4. 실행 (Jupyter 환경에서는 await 키워드를 바로 쓸 수 있습니다)
await test_streaming()

c:\dev2\tax_chatbot\.venv\Lib\site-packages\langchain_tavily\tavily_research.py:97: UserWarning: Field name "output_schema" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]
c:\dev2\tax_chatbot\.venv\Lib\site-packages\langchain_tavily\tavily_research.py:97: UserWarning: Field name "stream" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]


🔴 스트리밍 시작...
소득세는 개인이나 법인이 벌어들인 소득에 대해 부과되는 세금입니다. 소득의 종류에는 이자소득, 배당소득, 사업소득, 근로소득, 연금소득, 기타소득, 퇴직소득, 양도소득 등이 포함됩니다. 과세 대상 소득에 대해 일정 비율로 세금을 납부하는 것이 소득세입니다.
🔵 스트리밍 종료


In [2]:
from openai import OpenAI

client = OpenAI()
models = client.models.list()

for m in models.data:
    print(m.id)

gpt-4-0613
gpt-4
gpt-3.5-turbo
gpt-5.2-codex
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
chatgpt-image-latest
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-4-1106-preview
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-3.5-turbo-0125
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
chatgpt-4o-latest
gpt-4o-audio-preview
gpt-4o-realtime-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-realtime-preview-2024-12-17
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-realtime-preview
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt